In [1]:
# import some data

import mne
import matplotlib.pyplot as plt
import scipy

In [2]:
%matplotlib qt5

In [11]:
raw=mne.io.read_raw_eeglab('trial_data_for_mri_cwl_development_cwldata.set')


The following EEG sensors did not have a position specified in the selected montage: ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6']. Their position has been left untouched.
The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
Events like the following will be dropped entirely: ['fl', 'R', 'boundary', 'eec', '[]'], 10 in total
709/1015 event codes could not be mapped to integers. Use the 'event_id' parameter to map such events manually.
96 events will be dropped because they occur on the same time sample as another event. `mne.io.Raw` objects store events on an event channel, which cannot represent two events on the same sample. You can extract the original event structure using `mne.io.eeglab.read_events_eeglab`. Then, you can e.g. subset the extracted events for constructing epochs.
Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file


<ipython-input-11-2a8b57b8448c>:1: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6']. Their position has been left untouched.
  raw=mne.io.read_raw_eeglab('trial_data_for_mri_cwl_development_cwldata.set')
<ipython-input-11-2a8b57b8448c>:1: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw=mne.io.read_raw_eeglab('trial_data_for_mri_cwl_development_cwldata.set')
<ipython-input-11-2a8b57b8448c>:1: RuntimeWarning: Events like the following will be dropped entirely: ['fl', 'R', 'boundary', 'eec', '[]'], 10 in total
  raw=mne.io.read_raw_eeglab('trial_data_for_mri_cwl_development_cwldata.set')
<ipython-input-11-2a8b57b8448c>:1: RuntimeWarning: 709/1015 event codes could not be mapped to integers. Use the 'event_id' parameter to map such events manually.
  raw=mne.io.read_raw_eeglab('tri

In [88]:
raw.info['chs'][32:36]

[{'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
  'unit_mul': 0,
  'range': 1.0,
  'cal': 1e-06,
  'kind': 2,
  'coil_type': 1,
  'unit': 107,
  'coord_frame': 0,
  'ch_name': 'CW1',
  'scanno': 33,
  'logno': 33},
 {'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
  'unit_mul': 0,
  'range': 1.0,
  'cal': 1e-06,
  'kind': 2,
  'coil_type': 1,
  'unit': 107,
  'coord_frame': 0,
  'ch_name': 'CW2',
  'scanno': 34,
  'logno': 34},
 {'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
  'unit_mul': 0,
  'range': 1.0,
  'cal': 1e-06,
  'kind': 2,
  'coil_type': 1,
  'unit': 107,
  'coord_frame': 0,
  'ch_name': 'CW3',
  'scanno': 35,
  'logno': 35},
 {'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
  'unit_mul': 0,
  'range': 1.0,
  'cal': 1e-06,
  'kind': 2,
  'coil_type': 1,
  'unit': 107,
  'coord_frame': 0,
  'ch_name': 'CW4',
  'scanno': 36,
  'logno': 36}]

In [33]:
cwl=raw[32:38,:][0]

In [35]:
cwl.shape

(6, 293241)

In [90]:
raw[:,:][0].shape

(39, 293241)

In [99]:
eeg=raw[10,:][0].flatten()

In [101]:
eeg.shape

(293241,)

In [26]:
pa.filters.FilterGNGD?

Init signature: pa.filters.FilterGNGD(n, mu=1.0, eps=1.0, ro=0.1, w='random')
Docstring:     
Adaptive GNGD filter.

**Args:**

* `n` : length of filter (integer) - how many input is input array
  (row of input matrix)

**Kwargs:**

* `mu` : learning rate (float). Also known as step size.
  If it is too slow,
  the filter may have bad performance. If it is too high,
  the filter will be unstable. The default value can be unstable
  for ill-conditioned input data.

* `eps` : compensation term (float) at the beginning. It is adaptive
  parameter.

* `ro` : step size adaptation parameter (float) at the beginning.
  It is adaptive parameter.

* `w` : initial weights of filter. Possible values are:
    
    * array with initial weights (1 dimensional array) of filter size

    * "random" : create random weights
    
    * "zeros" : create zero value weights
File:           ~/.conda/envs/rt/lib/python3.6/site-packages/padasip/filters/gngd.py
Type:           type


In [25]:
import numpy as np
import matplotlib.pylab as plt
import padasip as pa 

In [143]:
NBCHAN=6
TIMEPOINTS=50

# these two function supplement your online measurment
def measure_x():
    # it produces input vector of size 3
    x = np.random.random(3)
    return x
    
def measure_d(x):
    # meausure system output
    d = 2*x[0] + 1*x[1] - 1.5*x[2]
    return d


N = 10000 # 100 samples
STARTSAMPLE = 40000
log_d = np.zeros(N)
log_y = np.zeros(N)
filt = pa.filters.FilterGNGD(NBCHAN*TIMEPOINTS, mu=0.5)
x=np.empty((NBCHAN*TIMEPOINTS))

for k in range(N):
    # the CWL data, yield it in steps of 100

    for i in range(TIMEPOINTS):
        b=i*6
        e=b+6
        x[b:e] = cwl[:,STARTSAMPLE-i+k]
        
    # predict new value - current sample
    y = filt.predict(x)

    
    d = eeg[STARTSAMPLE+k]
    rest = d - y
    

    filt.adapt(d, x)
    # log values
    log_d[k] = d
    log_y[k] = y
    
### show results
plt.figure(figsize=(15,9))
plt.subplot(211);plt.title("Adaptation");plt.xlabel("samples - k")
plt.plot(log_d,"b", label="d - target")
plt.plot(log_y,"g", label="y - output");plt.legend()
plt.subplot(212);plt.title("Filter error");plt.xlabel("samples - k")
plt.plot(10*np.log10((log_d-log_y)**2),"r", label="e - error [dB]")
plt.legend(); plt.tight_layout(); plt.show()

In [149]:
plt.gca().get_xlim()

(103206.21553810008, 105799.90325468284)

In [151]:
plt.gca().set_xlim((103206.21553810008, 105799.90325468284))

(103206.21553810008, 105799.90325468284)

In [42]:
x=np.empty((1,NBCHAN*TIMEPOINTS))

In [28]:
# these two function supplement your online measurment
def measure_x():
    # it produces input vector of size 3
    x = np.random.random(3)
    return x
    
def measure_d(x):
    # meausure system output
    d = 2*x[0] + 1*x[1] - 1.5*x[2]
    return d
    
N = 100
log_d = np.zeros(N)
log_y = np.zeros(N)
filt = pa.filters.FilterGNGD(3, mu=1.)
for k in range(N):
    # measure input
    x = measure_x()
    # predict new value
    y = filt.predict(x)
    # do the important stuff with prediction output
    pass    
    # measure output
    d = measure_d(x)
    # update filter
    filt.adapt(d, x)
    # log values
    log_d[k] = d
    log_y[k] = y
    
### show results
plt.figure(figsize=(15,9))
plt.subplot(211);plt.title("Adaptation");plt.xlabel("samples - k")
plt.plot(log_d,"b", label="d - target")
plt.plot(log_y,"g", label="y - output");plt.legend()
plt.subplot(212);plt.title("Filter error");plt.xlabel("samples - k")
plt.plot(10*np.log10((log_d-log_y)**2),"r", label="e - error [dB]")
plt.legend(); plt.tight_layout(); plt.show()

In [31]:
filt.w

array([ 1.98772603,  0.99780143, -1.48700121])